In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import ranksums, ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from chart_studio import plotly
import plotly.graph_objects as go
import plotly.express as px

import os
import json

In [12]:
with open('data/paths.json','r') as f:
    paths = json.load(f)

data_path = paths["data_path"]

In [13]:
data = pd.read_csv(os.path.join(data_path,'clean_data.csv'))
data.head()

,sampleNameOnTube,gender,treatment,tissue,injectionVolume,componentName,massInfo,componentGroupName,expectedRt,area,isArea,areaRatio,retentionTime,signalNoise,tissueWeightMg,normalizedArea,sampleName,isNormalizationFactor,normfactor
0,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H_d9_SPLASH.IS,738.7 / 184.2,SM,12.51,6.142877e+07,NaN,NaN,12.514791,141.426257,56,NaN,20,True,10
1,C20 M1 S,Male,Control,Striatum,10,SM(14:0)+H,675.5 / 184.1,SM,12.51,4.116176e+06,6.142877e+07,0.067007,12.737513,77.971430,56,0.119656,20,False,10
2,C20 M1 S,Male,Control,Striatum,10,SM(16:0)+H,703.6 / 184.1,SM,12.51,2.433723e+07,6.142877e+07,0.396186,12.662914,147.602022,56,0.707475,20,False,10
3,C20 M1 S,Male,Control,Striatum,10,SM(18:0)+H,731.6 / 184.1,SM,12.51,1.432478e+08,6.142877e+07,2.331933,12.580159,144.222304,56,4.164166,20,False,10
4,C20 M1 S,Male,Control,Striatum,10,SM(18:1)+H,729.6 / 184.1,SM,12.51,1.268323e+08,6.142877e+07,2.064706,12.572303,193.985512,56,3.686974,20,False,10


In [30]:
data['componentGroupName'].unique()

array(['SM', 'CE', 'CER', 'DCER', 'HCER', 'LCER', 'TAG', 'DAG', 'MAG',
       'LPC', 'PC', 'LPE', 'PE', 'PE-O', 'PE-P', 'LPG', 'PG', 'LPI', 'PI',
       'LPS', 'PS', 'FFA', 'PUFA'], dtype=object)

### Descriptive Analysis

In [15]:
# create figure
fig = go.Figure()
buttons = [{"label": "Select Lipid: ", "method": "update", "args": [{'visible': [False for cn in data[data['isNormalizationFactor'] == False]['componentName'].unique()]}]}]

for cn in data[data['isNormalizationFactor'] == False]['componentName'].unique():
    sum_of_normalized_area_ratio = dict(np.sum(data[(data['isNormalizationFactor'] == False] & data['componentName'] == cn)]))
    
    trace = go.Bar(x=list(sum_of_normalized_area_ratio.keys()), y=list(sum_of_normalized_area_ratio.values()),
                   name=team + " win/loss percent", visible=False, hoverinfo="text",
                   hovertext=[f"{team} VS. {tm1}</br>"
                              f"</br>Wins: {wins_dict[tm1]}"
                              f"</br>Losses: {losses_dict[tm1]}"
                              f"</br>Total:{tot_games[tm1]}"
                              f"</br>Win/Loss Pct: {pct_winloss[tm1]}"
                              for tm1 in teams],
                   marker_color=win_loss_colors)
    
    btnx = {"label": tm, 'method': 'update',
            "args": [{'visible': [True if tm == tmx else False for tmx in teams]}]}

fig.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)

In [16]:
f = dict(50)

TypeError: 'int' object is not iterable

In [17]:
list_of_lypids = list(data[data['isNormalizationFactor'] == False]['componentName'].unique())
list_of_lypids[0:10]

['SM(14:0)+H',
 'SM(16:0)+H',
 'SM(18:0)+H',
 'SM(18:1)+H',
 'SM(20:0)+H',
 'SM(20:1)+H',
 'SM(22:0)+H',
 'SM(22:1)+H',
 'SM(24:0)+H',
 'SM(24:1)+H']

In [32]:
sum_normalized_area_dict = {}
for elm in list_of_lypids:
    subjects = set(data[(data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['sampleNameOnTube'])
    sum_normalized_area_dict.update({elm: {x: {'sumNormalizedArea':np.sum(data[(data['sampleNameOnTube'] == x) & (data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['normalizedArea']),
                                              'sumAreaRatio':np.sum(data[(data['sampleNameOnTube'] == x) & (data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['areaRatio']),
                                              'lypidClass':data[(data['sampleNameOnTube'] == x) & (data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['componentGroupName']} for x in subjects}})
    

AttributeError: 'Series' object has no attribute 'value'

In [31]:
df_sum_normnalize_area = pd.DataFrame(sum_normalized_area_dict)
df_sum_normnalize_area.iloc[1,1]

{'sumNormalizedArea': 1.0243914616798442,
 'sumAreaRatio': 0.532683560073519,
 'lypidClass': 16704    SM
 Name: componentGroupName, dtype: object}

In [24]:
sum_area_ratio_dict = {}
for elm in list_of_lypids:
    subjects = set(data[(data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['sampleNameOnTube'])
    sum_area_ratio_dict.update({elm: {x: np.sum(data[(data['sampleNameOnTube'] == x) & (data['isNormalizationFactor'] == False) & (data['componentName'] == elm)]['areaRatio']) for x in subjects}})

In [20]:
df_normalized_area_ratio = pd.DataFrame(sum_normalized_area_dict)
df_normalized_area_ratio

,SM(14:0)+H,SM(16:0)+H,SM(18:0)+H,SM(18:1)+H,SM(20:0)+H,SM(20:1)+H,SM(22:0)+H,SM(22:1)+H,SM(24:0)+H,SM(24:1)+H,...,"FFA 24:1 (Nervonic acid, nâ€“9)",FFA 25:0 (Pentacosanoic acid),FFA 26:0 (Cerotic acid),FFA 27:0 (Heptacosanoic acid),FFA 28:0 (Montanic acid),EPA,DHA,DPA,Adrenic Acid,AA
T25 M1 S,"[0.0807521125419294, 0.0419910985218033]","[0.6842327013039231, 0.35580100467804]","[5.671724096616038, 2.94929653024034]","[4.2014515508634425, 2.18475480644899]","[0.2289577023349076, 0.119058005214152]","[0.0858082793076682, 0.0446203052399875]","[0.0, 0.0]","[0.107508689953359, 0.0559045187757467]","[0.1856347201109307, 0.096530054457684]","[0.7351435419537807, 0.382274641815966]",...,"[250.7063058436865, 130.367279038717]","[301.4249143261885, 156.740955449618]","[171.81399782505116, 89.3432788690266]","[140.57606978067173, 73.0995562859493]","[153.01770203122518, 79.5692050562371]","[21.69913321439269, 11.2835492714842]","[2447.1146418686158, 1272.49961377168]","[187.94364589165383, 97.73069586366]","[41.75710216639173, 21.7136931265237]","[2687.1750221710768, 1397.33101152896]"
C11 M2 C,"[0.0930060423687609, 0.0483631420317557]","[1.0243914616798442, 0.532683560073519]","[8.407758491255885, 4.37203441545306]","[4.187029115440674, 2.17725514002915]","[0.2799184419244673, 0.145557589800723]","[0.0896338832450053, 0.0466096192874028]","[0.2757893478903442, 0.143410460902979]","[0.26531222560385, 0.137962357314002]","[0.590883511518452, 0.307259425989595]","[1.5087305650118346, 0.784539893806154]",...,"[26.213334217460385, 13.6309337930794]","[31.94536208454769, 16.6115882839648]","[28.94610337459135, 15.0519737547875]","[16.389940151351404, 8.52276887870273]","[113.3446174782596, 58.939201088695]","[2.7187933870252112, 1.41377256125311]","[251.47987664931728, 130.769535857645]","[20.107671328580576, 10.4559890908619]","[5.301340922816962, 2.75669727986482]","[278.7053501583924, 144.926782082364]"
C17 F1 C,"[0.0679079331677072, 0.0421029185639785]","[0.4358647865052678, 0.270236167633266]","[2.913561562743113, 1.80640816890073]","[2.286490268020887, 1.41762396617295]","[0.0486880991743595, 0.0301866214881029]","[0.012386875526243, 0.0076798628262707]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.3375867609148403, 0.209303791767201]",...,"[73.17777230468161, 45.3702188289026]","[134.9895498396213, 83.6935209005652]","[67.97967252933161, 42.1473969681856]","[68.7005808496084, 42.5943601267572]","[175.36057932216127, 108.72355917974]","[6.44154508458142, 3.99375795244048]","[1203.4960508217307, 746.167551509473]","[87.91021063181257, 54.5043305917238]","[13.53573207980113, 8.3921538894767]","[1044.437974944466, 647.551544465569]"
C20 M1 S,"[0.1196558824120087, 0.0670072941507249]","[0.7074752287752839, 0.396186128114159]","[4.164165901046803, 2.33193290458621]","[3.686974338684893, 2.06470562966354]","[0.1575384622346098, 0.0882215388513815]","[0.1874134230168053, 0.104951516889411]","[0.0, 0.0]","[0.1306225339010862, 0.0731486189846083]","[0.1920515503729553, 0.107548868208855]","[0.6201379652200232, 0.347277260523213]",...,"[24.934691623146605, 13.9634273089621]","[24.23737515782357, 13.5729300883812]","[12.264948109759, 6.86837094146504]","[18.526593031324108, 10.3748920975415]","[3.429107670942661, 1.92030029572789]","[0.7934247553452303, 0.444317862993329]","[99.62489609648, 55.7899418140288]","[7.934364103168447, 4.44324389777433]","[2.3069467710770715, 1.29189019180316]","[129.7085975167875, 72.636814609401]"
T34 M2 S,"[0.0721122679945827, 0.0598531824355037]","[0.379575884185206, 0.315047983873721]","[1.97652111357994, 1.64051252427135]","[2.1965169730105782, 1.82310908759878]","[0.040326565956445, 0.0334710497438494]","[0.0533136831181792, 0.0442503569880888]","[0.0, 0.0]","[0.0812147463810444, 0.0674082394962669]","[0.024723599922445, 0.0205205879356294]","[0.1791318353505349, 0.148679423340944]",...,"[721.9264310119096, 599.198937739885]","[770.1708025731589, 639.241766135722]","[692.4530876249927, 574.736062728744]","[479.7263

In [28]:
df_sum_normnalize_area_t = df_sum_normnalize_area.transpose()
df_sum_normnalize_area_t.head()

,T25 M1 S,C11 M2 C,C17 F1 C,C20 M1 S,T34 M2 S,T33 F2 C,C18 M1 C,T21 F2 C,C20 M1 C,C21 M1 S,...,T24 M2 C,C19 F1 C,C19 M2 C,C22 F2 S,T20 F1 S,T18 F2 C,T23 M1 S,T21 M1 S,C22 M2 S,C19 M2 S
SM(14:0)+H,"{'sumNormalizedArea': 0.0807521125419294, 'sum...","{'sumNormalizedArea': 0.0930060423687609, 'sum...","{'sumNormalizedArea': 0.0679079331677072, 'sum...","{'sumNormalizedArea': 0.1196558824120087, 'sum...","{'sumNormalizedArea': 0.0721122679945827, 'sum...","{'sumNormalizedArea': 0.075944336739685, 'sumA...","{'sumNormalizedArea': 0.0519529995572229, 'sum...","{'sumNormalizedArea': 0.0291574666315785, 'sum...","{'sumNormalizedArea': 0.0740308392477825, 'sum...","{'sumNormalizedArea': 0.1382286448633851, 'sum...",...,"{'sumNormalizedArea': 0.047548951570821, 'sumA...","{'sumNormalizedArea': 0.0639145830942548, 'sum...","{'sumNormalizedArea': 0.0526652285276708, 'sum...","{'sumNormalizedArea': 0.0813561397557547, 'sum...","{'sumNormalizedArea': 0.081463192453759, 'sumA...","{'sumNormalizedArea': 0.0709827372595397, 'sum...","{'sumNormalizedArea': 0.0706047933043062, 'sum...","{'sumNormalizedArea': 0.0669834959357647, 'sum...","{'sumNormalizedArea': 0.0847692228904054, 'sum...","{'sumNormalizedArea': 0.0670837769341967, 'sum..."
SM(16:0)+H,"{'sumNormalizedArea': 0.6842327013039231, 'sum...","{'sumNormalizedArea': 1.0243914616798442, 'sum...","{'sumNormalizedArea': 0.4358647865052678, 'sum...","{'sumNormalizedArea': 0.7074752287752839, 'sum...","{'sumNormalizedArea': 0.379575884185206, 'sumA...","{'sumNormalizedArea': 0.4060395875782215, 'sum...","{'sumNormalizedArea': 0.3845699963294983, 'sum...","{'sumNormalizedArea': 0.3261375518175199, 'sum...","{'sumNormalizedArea': 0.5302688442413063, 'sum...","{'sumNormalizedArea': 0.9260204445136708, 'sum...",...,"{'sumNormalizedArea': 0.3221735780334912, 'sum...","{'sumNormalizedArea': 0.643342252839861, 'sumA...","{'sumNormalizedArea': 0.3908013301608161, 'sum...","{'sumNormalizedArea': 0.4040666697549857, 'sum...","{'sumNormalizedArea': 0.5245482585215061, 'sum...","{'sumNormalizedArea': 0.827878588458355, 'sumA...","{'sumNormalizedArea': 0.446967933231544, 'sumA...","{'sumNormalizedArea': 0.6407051376741213, 'sum...","{'sumNormalizedArea': 0.5229581817302751, 'sum...","{'sumNormalizedArea': 0.4872427368315819, 'sum..."
SM(18:0)+H,"{'sumNormalizedArea': 5.671724096616038, 'sumA...","{'sumNormalizedArea': 8.407758491255885, 'sumA...","{'sumNormalizedArea': 2.913561562743113, 'sumA...","{'sumNormalizedArea': 4.164165901046803, 'sumA...","{'sumNormalizedArea': 1.97652111357994, 'sumAr...","{'sumNormalizedArea': 2.1704206490223723, 'sum...","{'sumNormalizedArea': 1.950912774735984, 'sumA...","{'sumNormalizedArea': 3.1099181609860542, 'sum...","{'sumNormalizedArea': 2.8806307219209377, 'sum...","{'sumNormalizedArea': 6.162486388947323, 'sumA...",...,"{'sumNormalizedArea': 1.7985876211769558, 'sum...","{'sumNormalizedArea': 5.66045594666956, 'sumAr...","{'sumNormalizedArea': 2.026730509644016, 'sumA...","{'sumNormalizedArea': 2.297432348147572, 'sumA...","{'sumNormalizedArea': 3.0580860988157124, 'sum...","{'sumNormalizedArea': 7.13550158568675, 'sumAr...","{'sumNormalizedArea': 3.12158800652116, 'sumAr...","{'sumNormalizedArea': 5.436465181121132, 'sumA...","{'sumNormalizedArea': 2.9005157983389838, 'sum...","{'sumNormalizedArea': 2.6485818804080163, 'sum..."
SM(18:1)+H,"{'sumNormalizedArea': 4.2014515508634425, 'sum...","{'sumNormalizedArea': 4.187029115440674, 'sumA...","{'sumNormalizedArea': 2.286490268020887, 'sumA...","{'sumNormalizedArea': 3.686974338684893, 'sumA...","{'sumNormalizedArea': 2.1965169730105782, 'sum...","{'sumNormalizedArea': 3.172520289458745, 'sumA...","{'sumNormalizedArea': 3.0736116340554336, 'sum...","{'sumNormalizedArea': 1.9134878021849089, 'sum...","{'sumNormalizedArea': 3.228950353443641, 'sumA...","{'sumNormalizedArea': 4.845863805373742, 'sumA...",...,"{'sumNormalizedArea': 2.466937018848721, 'sumA...","{'sumNormalizedArea': 3.7535370211433903, 'sum...","{'sumNorma

In [21]:
df_normalized_area_ratio_t = df_normalized_area_ratio.transpose()
df_normalized_area_ratio_t.head()

,T25 M1 S,C11 M2 C,C17 F1 C,C20 M1 S,T34 M2 S,T33 F2 C,C18 M1 C,T21 F2 C,C20 M1 C,C21 M1 S,...,T24 M2 C,C19 F1 C,C19 M2 C,C22 F2 S,T20 F1 S,T18 F2 C,T23 M1 S,T21 M1 S,C22 M2 S,C19 M2 S
SM(14:0)+H,"[0.0807521125419294, 0.0419910985218033]","[0.0930060423687609, 0.0483631420317557]","[0.0679079331677072, 0.0421029185639785]","[0.1196558824120087, 0.0670072941507249]","[0.0721122679945827, 0.0598531824355037]","[0.075944336739685, 0.0387316117372394]","[0.0519529995572229, 0.0311717997343338]","[0.0291574666315785, 0.0160366066473682]","[0.0740308392477825, 0.0473797371185808]","[0.1382286448633851, 0.0428508799076494]",...,"[0.047548951570821, 0.0323332870681583]","[0.0639145830942548, 0.0262049790686445]","[0.0526652285276708, 0.0326524416871559]","[0.0813561397557547, 0.0512543680461255]","[0.081463192453759, 0.053765707019481]","[0.0709827372595397, 0.0283930949038159]","[0.0706047933043062, 0.0529535949782297]","[0.0669834959357647, 0.0408599325208165]","[0.0847692228904054, 0.0508615337342433]","[0.0670837769341967, 0.04092110392986]"
SM(16:0)+H,"[0.6842327013039231, 0.35580100467804]","[1.0243914616798442, 0.532683560073519]","[0.4358647865052678, 0.270236167633266]","[0.7074752287752839, 0.396186128114159]","[0.379575884185206, 0.315047983873721]","[0.4060395875782215, 0.207080189664893]","[0.3845699963294983, 0.230741997797699]","[0.3261375518175199, 0.179375653499636]","[0.5302688442413063, 0.339372060314436]","[0.9260204445136708, 0.287066337799238]",...,"[0.3221735780334912, 0.219078033062774]","[0.643342252839861, 0.263770323664343]","[0.3908013301608161, 0.242296824699706]","[0.4040666697549857, 0.254562001945641]","[0.5245482585215061, 0.346201850624194]","[0.827878588458355, 0.331151435383342]","[0.446967933231544, 0.335225949923658]","[0.6407051376741213, 0.390830133981214]","[0.5229581817302751, 0.313774909038165]","[0.4872427368315819, 0.297218069467265]"
SM(18:0)+H,"[5.671724096616038, 2.94929653024034]","[8.407758491255885, 4.37203441545306]","[2.913561562743113, 1.80640816890073]","[4.164165901046803, 2.33193290458621]","[1.97652111357994, 1.64051252427135]","[2.1704206490223723, 1.10691453100141]","[1.950912774735984, 1.17054766484159]","[3.1099181609860542, 1.71045498854233]","[2.8806307219209377, 1.8436036620294]","[6.162486388947323, 1.91037078057367]",...,"[1.7985876211769558, 1.22303958240033]","[5.66045594666956, 2.32078693813452]","[2.026730509644016, 1.25657291597929]","[2.297432348147572, 1.44738237933297]","[3.0580860988157124, 2.01833682521837]","[7.13550158568675, 2.8542006342747]","[3.12158800652116, 2.34119100489087]","[5.436465181121132, 3.31624376048389]","[2.9005157983389838, 1.74030947900339]","[2.6485818804080163, 1.61563494704889]"
SM(18:1)+H,"[4.2014515508634425, 2.18475480644899]","[4.187029115440674, 2.17725514002915]","[2.286490268020887, 1.41762396617295]","[3.686974338684893, 2.06470562966354]","[2.1965169730105782, 1.82310908759878]","[3.172520289458745, 1.61798534762396]","[3.0736116340554336, 1.84416698043326]","[1.9134878021849089, 1.0524182912017]","[3.228950353443641, 2.06652822620393]","[4.845863805373742, 1.50221777966586]",...,"[2.466937018848721, 1.67751717281713]","[3.7535370211433903, 1.53895017866879]","[3.061570874964661, 1.89817394247809]","[3.4052180378769203, 2.14528736386246]","[3.1608452235042424, 2.0861578475128]","[4.531003135665375, 1.81240125426615]","[2.45216576401056, 1.83912432300792]","[3.640064521240508, 2.22043935795671]","[3.6710852549046007, 2.20265115294276]","[3.286533902425754, 2.00478568047971]"
SM(20:0)+H,"[0.2289577023349076, 0.119058005214152]","[0.2799184419244673, 0.145557589800723]","[0.0486880991743595, 0.0301866214881029]","[0.1575384622346098, 0.0882215388513815]","[0.040326565956445, 0.0334710497438494]","[0.0704148334804611, 0.0359115650750352]","[0.0550112953569298, 0.0330067772141579]","[0.2390452807768545, 0.13147490442727]","[0.0965864951929185, 0.0618153569234679]","[0.239636772815978, 0.0742873995729532]",...,"[0.0419583059394183,

In [29]:
df_sum_normnalize_area_t.to_csv(os.path.join(data_path, 'sum_normalized_area_ratio_full.csv'))